In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image, ImageStat
import cv2
import os
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [2]:
class DehazingDataset(Dataset):
    def __init__(self, foggy_dir, bb_dir, transform=None):
        """
        Args:
            foggy_dir (string): Path to the directory with the foggy images.
            bb_dir (string): Path to the directory with bounding box annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.foggy_dir = os.path.join(foggy_dir, 'JPEGImages')
        self.bb_dir = os.path.join(bb_dir, 'Annotations')
        self.transform = transform
        self.images = [f for f in os.listdir(self.foggy_dir) if os.path.isfile(os.path.join(self.foggy_dir, f))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        foggy_image_path = os.path.join(self.foggy_dir, img_name)
        bb_file_path = os.path.join(self.bb_dir, img_name.replace('.png', '.xml').replace('.jpg', '.xml'))

        foggy_image = Image.open(foggy_image_path).convert('RGB')

        # Assuming bounding box annotations are in XML and need parsing
        bboxes = self.parse_bboxes(bb_file_path)

        sample = {'foggy_image': foggy_image, 'bboxes': bboxes}

        if self.transform:
            sample['foggy_image'] = self.transform(sample['foggy_image'])

        return sample

    def parse_bboxes(self, xml_path):
        # Implement XML parsing here
        import xml.etree.ElementTree as ET
        bboxes = []
        tree = ET.parse(xml_path)
        root = tree.getroot()
        for member in root.findall('object'):
            bboxes.append({
                'class_label': member.find('name').text,
                'x_center': float(member.find('bndbox/xmin').text),
                'y_center': float(member.find('bndbox/ymin').text),
                'width': float(member.find('bndbox/xmax').text) - float(member.find('bndbox/xmin').text),
                'height': float(member.find('bndbox/ymax').text) - float(member.find('bndbox/ymin').text)
            })
        return bboxes


In [3]:
from torch.utils.data.dataloader import default_collate
import torch

def collate_fn(batch):
    
    # Extract images and bounding boxes from the batch
    foggy_images = [item['foggy_image'] for item in batch]
    bboxes = [item['bboxes'] for item in batch]

    # Collate the images into a single tensor
    foggy_images = torch.stack(foggy_images, dim=0)

    # Bounding boxes are returned as is, since their sizes can vary and they cannot be stacked into a single tensor
    return {'foggy_image': foggy_images, 'bboxes': bboxes}



In [4]:
# Define the directory paths
foggy_dir = '/home/stu12/s11/ak1825/idai710/Project/RTTS'
bb_dir = '/home/stu12/s11/ak1825/idai710/Project/RTTS'

# Define transformations, if any
transform = transforms.Compose([
    transforms.Resize((640, 640)),  # Resize images to 640x640
    transforms.ToTensor()         # Convert images to PyTorch tensors
      # Normalize images
])

# Initialize the dataset
dataset = DehazingDataset(foggy_dir=foggy_dir, bb_dir=bb_dir, transform=transform)

# Initialize the DataLoader
data_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=4, collate_fn=collate_fn)

In [5]:
len(dataset)

4322

In [6]:
len(data_loader)

1081

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Adjust torch.load to map model to the correct device
model = torch.load('/home/stu12/s11/ak1825/idai710/Project/Dehazed-Detection/AOD-NetX/aodnetX2.pt', map_location=device)
model.eval()


DehazeNetAttention(
  (relu): ReLU(inplace=True)
  (e_conv1): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
  (e_conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e_conv3): Conv2d(6, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (e_conv4): Conv2d(6, 3, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (e_conv5): Conv2d(12, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (attention): SpatialAttentionLayer(
    (conv1): Conv2d(1, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (sigmoid): Sigmoid()
  )
)

In [8]:
def calculate_brightness(image):
    stat = ImageStat.Stat(image)
    r,g,b = stat.mean
    brightness_value = np.sqrt(0.299 * (r**2) + 0.587 * (g**2) + 0.114 * (b**2))
    return brightness_value

In [9]:
def calculate_contrast(image):
    image_gray = image.convert('L')
    return np.std(image_gray)

In [10]:
def calculate_color_shift(image):
    """Calculate the average distance of image colors from pure white."""
    # Convert the image into a NumPy array
    pixels = np.array(image)
    
    # Define the RGB value for pure white
    white = np.array([255, 255, 255])
    
    # Calculate the Euclidean distance from white for each pixel
    distances = np.sqrt(np.sum((pixels - white) ** 2, axis=-1))
    
    # Calculate the average distance
    avg_distance = np.mean(distances)
    
    # Normalize the average distance to a scale of 0 to 1 for consistency with other metrics
    # Assuming the maximum possible distance from white is the distance from white to black (sqrt(3) * 255)
    normalized_avg_distance = avg_distance / (np.sqrt(3) * 255)
    
    # Return the normalized average distance as the color shift metric
    return normalized_avg_distance

In [11]:
def normalize(value, min_value, max_value):
    """Normalize the value to a [0, 1] scale."""
    return (value - min_value) / (max_value - min_value)

def calculate_haze_index(image, min_brightness, max_brightness, min_contrast, max_contrast, min_color_shift, max_color_shift, weights={'brightness': 0.33, 'contrast': 0.33, 'color_shift': 0.34}):
    # Calculate metrics
    brightness = calculate_brightness(image)
    contrast = calculate_contrast(image)
    color_shift = calculate_color_shift(image)
    
    # Normalize metrics
    norm_brightness = normalize(brightness, min_brightness, max_brightness)
    norm_contrast = 1 - normalize(contrast, min_contrast, max_contrast) # Invert because higher contrast means less haze
    norm_color_shift = normalize(color_shift, min_color_shift, max_color_shift)
    
    # Calculate weighted haze index
    haze_index = (weights['brightness'] * norm_brightness +
                  weights['contrast'] * norm_contrast +
                  weights['color_shift'] * norm_color_shift)
    return haze_index


In [12]:
min_brightness, max_brightness = 24, 233
min_contrast, max_contrast = 8, 104
min_color_shift, max_color_shift = 0, 1

In [13]:
# import numpy as np

# def psnr(target, prediction):
#     mse = np.mean((target - prediction) ** 2)
#     if mse == 0:
#         return float('inf')
#     max_pixel = 255.0
#     return 20 * np.log10(max_pixel / np.sqrt(mse))


In [14]:
# total_psnr = 0.0
# num_images = 0

# with torch.no_grad():
#     for data in data_loader:
#         foggy_images = data['foggy_image'].to(device)
#         bounding_boxes = data['bboxes']  # Retrieve bounding boxes from data

#         # Call the model's forward method with both foggy images and bounding boxes
#         outputs = model(foggy_images, bounding_boxes)

#         # Move tensors to CPU for PSNR calculation
#         outputs = outputs.cpu().numpy()
#         foggy_images = foggy_images.cpu().numpy()

#         # Calculate PSNR for each image in the batch
#         for i in range(foggy_images.shape[0]):
#             # Convert tensors to proper scale if they were normalized (0-1 to 0-255)
#             original_image_scaled = (foggy_images[i] * 255).astype(np.uint8)
#             predicted_image_scaled = (outputs[i] * 255).astype(np.uint8)

#             current_psnr = psnr(original_image_scaled, predicted_image_scaled)
#             total_psnr += current_psnr
#             num_images += 1

# average_psnr = total_psnr / num_images
# print(f'Average PSNR: {average_psnr:.2f} dB')



In [17]:
total_haze_index_foggy = 0.0
total_haze_index_predicted = 0.0
num_images = 0

with torch.no_grad():
    for data in data_loader:
        foggy_images = data['foggy_image'].to(device)
        bounding_boxes = data['bboxes']  # Retrieve bounding boxes from data

        # Call the model's forward method with both foggy images and bounding boxes
        outputs = model(foggy_images, bounding_boxes)

        # Move tensors to CPU for haze index calculation
        outputs = outputs.cpu().numpy()
        foggy_images = foggy_images.cpu().numpy()

        # Calculate haze index for each image in the batch
        for i in range(foggy_images.shape[0]):
            # Convert tensors to proper scale if they were normalized (0-1 to 0-255)
            foggy_image = foggy_images[i].squeeze().transpose((1, 2, 0))
            predicted_image = outputs[i].squeeze().transpose((1, 2, 0))

            # Convert tensors to images
            foggy_image = (255 * foggy_image).astype(np.uint8)
            predicted_image = (255 * predicted_image).astype(np.uint8)

            foggy_image = Image.fromarray(foggy_image)
            predicted_image = Image.fromarray(predicted_image)

            # Calculate haze index for foggy image
            haze_index_foggy = calculate_haze_index(foggy_image, min_brightness, max_brightness, min_contrast, max_contrast, min_color_shift, max_color_shift)
            total_haze_index_foggy += haze_index_foggy

            # Calculate haze index for predicted image
            haze_index_predicted = calculate_haze_index(predicted_image, min_brightness, max_brightness, min_contrast, max_contrast, min_color_shift, max_color_shift)
            total_haze_index_predicted += haze_index_predicted

            num_images += 1

# Calculate average haze index for foggy and predicted images
average_haze_index_foggy = total_haze_index_foggy / num_images
average_haze_index_predicted = total_haze_index_predicted / num_images

print(f'Average Haze Index (Foggy): {average_haze_index_foggy:.2f}')
print(f'Average Haze Index (Predicted): {average_haze_index_predicted:.2f}')


Average Haze Index (Foggy): 0.53
Average Haze Index (Predicted): 0.54
